In [ ]:
!pip install kaggle
!pip install wordcloud

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.1/511.1 kB 6.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from wordcloud import WordCloud, STOPWORDS
import string
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MultiLabelBinarizer

import torch

In [ ]:
!kaggle datasets download -d niraliivaghani/flipkart-dataset

Dataset URL: https://www.kaggle.com/datasets/niraliivaghani/flipkart-dataset
License(s): ODbL-1.0
  0% 0.00/7.85M [00:00<?, ?B/s]
100% 7.85M/7.85M [00:00<00:00, 98.3MB/s]


In [ ]:
!unzip /content/flipkart-dataset.zip -d /content/flipkart-dataset

Archive:  /content/flipkart-dataset.zip
  inflating: /content/flipkart-dataset/Dataset.csv  


In [ ]:
df = pd.read_csv('/content/flipkart-dataset/Dataset.csv', encoding='ISO-8859-1')

<ipython-input-9-4fba2ae6ab62>:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/flipkart-dataset/Dataset.csv', encoding='ISO-8859-1')


In [ ]:
data1 = df[['Summary']]
data1.head()

,Summary
0,it's really worth every single penny. it works...
1,I bought Crompton Ozone 75 Desert Air Cooler i...
2,GREAT packaging by seller. As this was the mos...
3,Delivery was delayed by two days except this e...
4,A Good cooler by Crompton. The height of the c...


In [ ]:
data1 = data1.sample(n=80000, random_state=42)
print(data1.shape)

(80000, 1)


In [ ]:
data1 = data1.dropna(subset=['Summary'])
data1 = data1[data1['Summary'].str.strip() != '']
data1.reset_index(drop=True, inplace=True)
data1.count()

,0
Summary,79580


In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text)
    filtered_words = [lemmatizer.lemmatize(word.lower()) for word in word_tokens if word.lower() not in stop_words]
    return ' '.join(filtered_words)

data1 = data1[['Summary']]
data1['Summary'] = data1['Summary'].astype(str).fillna('')
data1['Processed_Summary'] = data1['Summary'].apply(preprocess_text)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
data1.head()

,Summary,Processed_Summary
0,Keep it up,keep
1,I gifted this box to my my beloved friend. He ...,gifted box beloved friend . like happy n't get...
2,It's gud to use...worth of money,'s gud use ... worth money
3,Good,good
4,Excellent,excellent


In [ ]:
data1.count()

,0
Summary,79580
Processed_Summary,79580


In [ ]:
!pip install yake

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.0/336.0 kB 11.0 MB/s eta 0:00:00


In [ ]:
nltk.download('words')

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [ ]:
from nltk.corpus import words
english_words = set(words.words())

In [ ]:
import yake
from nltk.corpus import words

nltk.download('words')
english_words = set(words.words())

yake_extractor = yake.KeywordExtractor(lan="en", n=1, top=6)

def extract_yake_keywords(text):
    keywords = yake_extractor.extract_keywords(text)
    valid_keywords = [keyword[0] for keyword in keywords if keyword[0].lower() in english_words]
    return valid_keywords

data1['Processed_Summary'] = data1['Processed_Summary'].astype(str).fillna('')

data1['Yake_Keywords'] = data1['Processed_Summary'].apply(extract_yake_keywords)


[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [ ]:
data1

,Summary,Processed_Summary,Yake_Keywords
0,Keep it up,keep,[]
1,I gifted this box to my my beloved friend. He ...,gifted box beloved friend . like happy n't get...,"[gifted, friend, box, beloved, happy]"
2,It's gud to use...worth of money,'s gud use ... worth money,"[gud, worth, money]"
3,Good,good,[good]
4,Excellent,excellent,[excellent]
...,...,...,...
79575,Very bad,bad,[bad]
79576,Best OTG oven ever seen,best otg oven ever seen,[oven]
79577,Good,good,[good]
79578,Fabulous,fabulous,[fabulous]


In [ ]:
X = data1["Processed_Summary"]
Y = data1["Yake_Keywords"]
data1["Yake_Keywords"].count()

79580

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.2, random_state=1
)

In [ ]:
!pip install gensim

In [ ]:
from gensim.models import Word2Vec

In [ ]:
mlb = MultiLabelBinarizer()
y_encoded = mlb.fit_transform(data1['Yake_Keywords'])

from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data1['Processed_Summary']).toarray()
vocab_size = len(vectorizer.vocabulary_)

# Convert to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y_encoded, dtype=torch.float32)

In [ ]:
# import torch.nn as nn
# import torch.optim as optim

# # Define the model architecture
# class KeywordExtractorModel(nn.Module):
#     def __init__(self, input_size, hidden_size, output_size):
#         super(KeywordExtractorModel, self).__init__()
#         self.fc1 = nn.Linear(input_size, hidden_size)
#         self.relu = nn.ReLU()
#         self.fc2 = nn.Linear(hidden_size, output_size)
#         self.sigmoid = nn.Sigmoid()

#     def forward(self, x):
#         x = self.fc1(x)
#         x = self.relu(x)
#         x = self.fc2(x)
#         return self.sigmoid(x)

# # Hyperparameters
# input_size = vocab_size
# hidden_size = 128  # You can adjust this
# output_size = len(mlb.classes_)  # Number of unique keywords

# # Initialize model, loss function, and optimizer
# model = KeywordExtractorModel(input_size, hidden_size, output_size)
# criterion = nn.BCEWithLogitsLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
# # Training parameters
# num_epochs = 10
# batch_size = 2

# # Create a DataLoader for batching
# from torch.utils.data import DataLoader, TensorDataset

# dataset = TensorDataset(X_tensor, y_tensor)
# dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# # Training loop
# for epoch in range(num_epochs):
#     total_loss = 0
#     count = 0;
#     for batch_X, batch_y in dataloader:
#         count+=1
#         if(count%5000==0):
#           print(count)
#         # Zero gradients
#         optimizer.zero_grad()

#         # Forward pass
#         outputs = model(batch_X)

#         # Compute loss
#         loss = criterion(outputs, batch_y)
#         total_loss += loss.item()

#         # Backward pass and optimize
#         loss.backward()
#         optimizer.step()

#     print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(dataloader)}")


In [ ]:
# # Example: Extract keywords from a new summary
# def predict_keywords(summary):
#     # Preprocess the summary using the same vectorizer
#     summary_vector = vectorizer.transform([summary]).toarray()
#     summary_tensor = torch.tensor(summary_vector, dtype=torch.float32)

#     # Predict using the model
#     with torch.no_grad():
#         output = model(summary_tensor)

#     # Convert output to keyword list based on a threshold (e.g., 0.5)
#     predicted_indices = (output > 0.00000000000001).squeeze().nonzero(as_tuple=True)[0]
#     predicted_keywords = [mlb.classes_[idx] for idx in predicted_indices]

#     return predicted_keywords

# # Test with a new summary
# new_summary = "The product has excellent sound quality and is very durable."
# pre_summ = preprocess_text(new_summary)
# print(pre_summ)
# predicted_keywords = predict_keywords(pre_summ)
# print("Extracted Keywords:", predicted_keywords)
# print(len(predict_keywords))


In [ ]:
# def predict_keywords(summary):
#     # Preprocess the summary using the same vectorizer
#     summary_vector = vectorizer.transform([summary]).toarray()
#     summary_tensor = torch.tensor(summary_vector, dtype=torch.float32)

#     # Predict using the model
#     with torch.no_grad():
#         output = model(summary_tensor)

#     # Convert output to a numpy array
#     output_np = output.squeeze().numpy()

#     # Create a dictionary of keywords and their scores (in percentage)
#     keyword_scores = {mlb.classes_[idx]: output_np[idx] * 100 for idx in range(len(output_np))}

#     # Filter out scores below a certain threshold (e.g., 0.01%)
#     keyword_scores = {k: v for k, v in keyword_scores.items() if v > 0.00000000001}  # Adjust threshold as needed

#     return keyword_scores

# # Test with a new summary
# new_summary = "The product has excellent sound quality and is very durable."
# pre_summ = preprocess_text(new_summary)
# print("Preprocessed Summary:", pre_summ)

# predicted_keywords_with_scores = predict_keywords(pre_summ)
# print("Extracted Keywords and Percentage Scores:", predicted_keywords_with_scores)
# print("Number of Extracted Keywords:", len(predicted_keywords_with_scores))


In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from sklearn.model_selection import train_test_split

def preprocess_data(df):
    processed_data = []
    for _, row in df.iterrows():
        sentence = row['Processed_Summary'].split()
        keywords = set(row['Yake_Keywords'])
        labels = [1 if word in keywords else 0 for word in sentence]
        processed_data.append((sentence, labels))
    return processed_data

def create_vocab(data):
    vocab = set()
    for sentence, _ in data:
        vocab.update(sentence)
    word_to_idx = {word: idx+1 for idx, word in enumerate(vocab)}  # 0 is reserved for padding
    return word_to_idx

def sentence_to_indices(sentence, word_to_idx):
    return [word_to_idx.get(word, 0) for word in sentence]

class KeywordDataset(Dataset):
    def __init__(self, data, word_to_idx):
        self.data = data
        self.word_to_idx = word_to_idx

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sentence, labels = self.data[idx]
        sentence_indices = sentence_to_indices(sentence, self.word_to_idx)
        return torch.tensor(sentence_indices, dtype=torch.long), torch.tensor(labels, dtype=torch.float)

def collate_fn(batch):
    sentences, labels = zip(*batch)
    sentences_padded = pad_sequence(sentences, batch_first=True, padding_value=0)
    labels_padded = pad_sequence(labels, batch_first=True, padding_value=0)
    return sentences_padded, labels_padded

processed_data = preprocess_data(data1)
word_to_idx = create_vocab(processed_data)

train_data, val_data = train_test_split(processed_data, test_size=0.2, random_state=42)

train_dataset = KeywordDataset(train_data, word_to_idx)
val_dataset = KeywordDataset(val_data, word_to_idx)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)

In [ ]:
import torch
import torch.nn as nn

class KeywordExtractionModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers, dropout):
        super(KeywordExtractionModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=num_layers,
                            batch_first=True, bidirectional=True, dropout=dropout)
        self.fc1 = nn.Linear(hidden_dim * 2, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        embedded = self.embedding(x)
        output, _ = self.lstm(embedded)
        output = self.dropout(output)
        output = torch.relu(self.fc1(output))
        output = self.fc2(output)
        return torch.sigmoid(output.squeeze(-1))

vocab_size = len(word_to_idx) + 1
embedding_dim = 100
hidden_dim = 128
num_layers = 2
dropout = 0.3

model = KeywordExtractionModel(vocab_size, embedding_dim, hidden_dim, num_layers, dropout)

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
import os
os.makedirs('model_checkpoints', exist_ok=True)

In [ ]:
import torch.nn.functional as F

def train(model, train_loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    for sentences, labels in train_loader:
        sentences, labels = sentences.to(device), labels.to(device)
        optimizer.zero_grad()
        output = model(sentences)
        loss = criterion(output, labels.float())
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)

def evaluate(model, val_loader, criterion, device):
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for sentences, labels in val_loader:
            sentences, labels = sentences.to(device), labels.to(device)
            output = model(sentences)
            loss = criterion(output, labels.float())
            total_loss += loss.item()
            preds = (output > 0.5).float()
            all_preds.extend(preds.cpu().numpy().flatten())
            all_labels.extend(labels.cpu().numpy().flatten())

    accuracy = sum(p == l for p, l in zip(all_preds, all_labels)) / len(all_preds)
    return total_loss / len(val_loader), accuracy

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

num_epochs = 10
best_val_loss = float('inf')
for epoch in range(num_epochs):
    train_loss = train(model, train_loader, optimizer, criterion, device)
    val_loss, val_accuracy = evaluate(model, val_loader, criterion, device)
    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train Loss: {train_loss:.4f}")
    print(f"Validation Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.4f}")


    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'train_loss': train_loss,
        'val_loss': val_loss,
        'val_accuracy': val_accuracy
    }, f'model_checkpoints/model_epoch_{epoch+1}.pth')

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'train_loss': train_loss,
            'val_loss': val_loss,
            'val_accuracy': val_accuracy
        }, 'model_checkpoints/best_model.pth')

    print(f"Model saved as model_checkpoints/model_epoch_{epoch+1}.pth")
    print()

def predict_keywords(model, sentence, word_to_idx, device):
    model.eval()
    indices = torch.tensor([word_to_idx.get(word, 0) for word in sentence]).unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(indices)
    predictions = (output > 0.5).float().squeeze().cpu().numpy()
    keywords = [word for word, is_keyword in zip(sentence, predictions) if is_keyword]
    return keywords

new_sentence = "The service was excellent and the product quality was outstanding"
predicted_keywords = predict_keywords(model, new_sentence.split(), word_to_idx, device)
print(f"Predicted keywords: {predicted_keywords}")

KeyboardInterrupt: 

In [ ]:
reviews = [
    "The battery life of this product is amazing; it lasts all day without charging.",
    "I found the user interface to be very intuitive and easy to navigate.",
    "This gadget has all the features I need, and it's very lightweight.",
    "Customer support was responsive and resolved my issue quickly.",
    "The sound quality is exceptional, providing a rich audio experience.",
    "I was disappointed with the delivery time; it took longer than expected.",
    "This product exceeded my expectations, especially in terms of performance.",
    "The design is sleek and modern, making it a great addition to my home.",
    "I love how versatile this product is; I can use it for multiple tasks.",
    "The quality of materials used is top-notch, ensuring durability."
]

for review in reviews:
    predicted_keywords = predict_keywords(model, review.split(), word_to_idx, device)
    print(f"Review: {review}")
    print(f"Predicted keywords: {predicted_keywords}\n")


In [ ]:
def load_model(model, optimizer, checkpoint_path):
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    train_loss = checkpoint['train_loss']
    val_loss = checkpoint['val_loss']
    val_accuracy = checkpoint['val_accuracy']

    return model, optimizer, epoch, train_loss, val_loss, val_accuracy

# Example usage:
model, optimizer, start_epoch, train_loss, val_loss, val_accuracy = load_model(model, optimizer, '/content/best_model.pth')
print(f"Loaded model from epoch {start_epoch} with validation accuracy: {val_accuracy:.4f}")

Loaded model from epoch 1 with validation accuracy: 0.9912


<ipython-input-34-c4bd106e54db>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path)


In [ ]:
def predict_keywords(model, sentence, word_to_idx, device):
    model.eval()
    indices = torch.tensor([word_to_idx.get(word, 0) for word in sentence]).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(indices)

    scores = output.squeeze().cpu().numpy()

    word_scores = {word: score for word, score in zip(sentence, scores)}

    predicted_keywords = {word: score for word, score in word_scores.items() if score > 0.01}

    return predicted_keywords, word_scores

new_sentence = "If you are a student or a professional using laptop for day to day tasks and for lite photo and video editing , this is the perfect machine and this would also support Apple intelligence ."
predicted_keywords, all_scores = predict_keywords(model, new_sentence.split(), word_to_idx, device)

print(f"Predicted keywords with scores: {predicted_keywords}")
print(f"All scores: {all_scores}")


Predicted keywords with scores: {'student': 0.16273315, 'using': 0.028448591, 'day': 0.020817919, 'photo': 0.257404, 'video': 0.30773565, 'editing': 0.13510999, ',': 0.020875935, 'perfect': 0.019328754, 'machine': 0.012723887, 'intelligence': 0.033478655}
All scores: {'If': 1.6292026e-05, 'you': 2.2596842e-06, 'are': 1.3824294e-06, 'a': 6.9115816e-07, 'student': 0.16273315, 'or': 1.6180062e-06, 'professional': 0.0028680544, 'using': 0.028448591, 'laptop': 0.004060209, 'for': 1.367233e-05, 'day': 0.020817919, 'to': 7.086813e-05, 'tasks': 4.2750038e-05, 'and': 0.00073790975, 'lite': 2.9835104e-05, 'photo': 0.257404, 'video': 0.30773565, 'editing': 0.13510999, ',': 0.020875935, 'this': 0.00015876166, 'is': 5.6923705e-05, 'the': 2.7120592e-05, 'perfect': 0.019328754, 'machine': 0.012723887, 'would': 0.000607031, 'also': 0.00816274, 'support': 0.00063870155, 'Apple': 0.00030530294, 'intelligence': 0.033478655, '.': 0.008213169}
